In [8]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.edge.options import Options
import csv
import pandas as pd
import os
import requests
import re
import time

In [9]:
driver = webdriver.Edge()

In [44]:
# Chỉnh lại vị trí lưu khóa học
PATH0 = "C:\\Users\\victo\\Downloads\\Coursera"

# Chỉnh lại link của bài đầu tiên trong khóa học
# có thể sẽ cần đăng nhập bằng tay ở bước này để qua xác nhận không phải robot
# nếu bị lỗi thì thay đổi lại đường link ở đây bằng bài học cần tải
driver.get("https://www.coursera.org/learn/agile-atlassian-jira/supplement/Oy2DU/welcome")

In [56]:
PATH = ""
MODULE = ""
LESSON = ""

def clean_path(path):
        forbidden_chars = r'[<>:"/\\|?*.]'
        clean_path = re.sub(forbidden_chars, '', path)
        return clean_path

def getPath():
    MODULE = ""
    LESSON = ""
    PATH = PATH0

    elem = driver.find_element(By.CLASS_NAME, "rc-ItemNavBreadcrumbs")
    links = elem.find_elements(By.TAG_NAME, "a")
    MODULE = links[1].text
    LESSON = links[2].text

    MODULE = clean_path(MODULE)
    LESSON = clean_path(LESSON)

    PATH = PATH + "\\" + MODULE + "\\" + LESSON
    return PATH, MODULE, LESSON

def download_file(url, file_name, PATH):
    if not os.path.exists(PATH):
        os.makedirs(PATH)
    
    file_path = os.path.join(PATH, file_name)
    response = requests.get(url, stream="mp4" in url)
    
    if response.status_code == 200:
        mode = "wb"
        with open(file_path, mode) as file:
            if "mp4" in url:
                for chunk in response.iter_content(chunk_size=1024):
                    if chunk:
                        file.write(chunk)
            else:
                file.write(response.content)
        print(f"Tải xuống thành công tại: {file_path}")
    else:
        print(f"Không thể tải xuống file. Lỗi HTTP {response.status_code}")

def download_html(url, file_name, PATH):
    html_content = driver.page_source

    if not os.path.exists(PATH):
        os.makedirs(PATH)

    file_path = os.path.join(PATH, file_name)
    response = requests.get(url, stream="mp4" in url)

    with open(file_path, "w", encoding="utf-8") as file:
        file.write(html_content)
    print(f"Tải xuống thành công tại: {file_path}")

try:
    while(1):
        PATH, MODULE, LESSON = getPath()
        
        try:
            elem = driver.find_element(By.CLASS_NAME, "item-page-content")
            
            try:
                elem.find_element(By.CLASS_NAME, "cml-asset.cml-asset-pdf.cml-nonmedia-asset")
                link = elem.find_element(By.TAG_NAME, "a").get_attribute("href")
                download_file(link, f"{LESSON}.pdf", PATH)

                elems = driver.find_elements(By.CLASS_NAME, "cds-button-label")
                elem = next((e for e in elems if "Next" in e.text), elems[0])
                elem.click()
                time.sleep(3)
            except:
                download_html(link, f"{LESSON}.html", PATH)
                elems = driver.find_elements(By.CLASS_NAME, "cds-button-label")
                elem = next((e for e in elems if "Next" in e.text), elems[0])
                if("Next" not in elem.text): break
                elem.click()
                time.sleep(3)
        except:
            elems = driver.find_elements(By.CLASS_NAME, "cds-tab-wrapper")
            elems[2].click()
            elems = driver.find_element(By.CLASS_NAME, "cds-4.css-0.cds-6.cds-grid-item")
            links = elems.find_elements(By.TAG_NAME, "a")
            download_links = []
            type = ["mp4", "WebVTT", "txt"]
            for link in links:
                if any(sub in link.text for sub in type):
                    download_links.append(link.get_attribute("href"))
            qualities = {"360p": 360, "480p": 480, "720p": 720, "1080p": 1080}
            best_quality = max((resolution for link in download_links for quality, resolution in qualities.items() if quality in link), default=0)

            download_links = [link for link in download_links if not any(quality in link and resolution < best_quality for quality, resolution in qualities.items())]

            for url in download_links:
                if "mp4" in url:
                    download_file(url, f"{LESSON}.mp4", PATH)
                elif "vtt" in url:
                    download_file(url, "Subtitle.vtt", PATH)
                elif "txt" in url:
                    download_file(url, "Transcript.txt", PATH)
            elems = driver.find_elements(By.CLASS_NAME, "cds-button-label")
            elem = next((e for e in elems if "Next" in e.text), elems[0])
            elem.click()
            time.sleep(3)
    print('Đã tìm hết tất cả các trang!')
except NoSuchElementException:
    print('Đã tìm hết tất cả các trang!')

Tải xuống thành công tại: C:\Users\victo\Downloads\Coursera\Module 4\Module 4 Assessment\Module 4 Assessment.html
1
Đã tìm hết tất cả các trang!


In [ ]:
# # PDF
# elem = driver.find_element(By.CLASS_NAME, "rc-CML")
# link = elem.find_element(By.TAG_NAME, "a").get_attribute("href")

# download_file(link, "lab.pdf")


In [ ]:
# # Video
# elems = driver.find_elements(By.CLASS_NAME, "cds-tab-wrapper")
# elems[2].click()
# elems = driver.find_element(By.CLASS_NAME, "cds-4.css-0.cds-6.cds-grid-item")
# links = elems.find_elements(By.TAG_NAME, "a")
# download_links = []
# name = []
# type = ["mp4", "WebVTT", "txt"]
# for link in links:
#     if any(sub in link.text for sub in type):
#         download_links.append(link.get_attribute("href"))
#         # download_links.append(link)
# download_links

In [ ]:
# qualities = {"360p": 360, "480p": 480, "720p": 720, "1080p": 1080}
# best_quality = max((resolution for link in download_links for quality, resolution in qualities.items() if quality in link), default=0)

# download_links = [link for link in download_links if not any(quality in link and resolution < best_quality for quality, resolution in qualities.items())]
# print(best_quality)
# download_links


In [ ]:
# def download_file(url, file_name):
#     if not os.path.exists(PATH):
#         os.makedirs(PATH)
    
#     file_path = os.path.join(PATH, file_name)
#     response = requests.get(url, stream="mp4" in url)
    
#     if response.status_code == 200:
#         mode = "wb"
#         with open(file_path, mode) as file:
#             if "mp4" in url:
#                 for chunk in response.iter_content(chunk_size=1024):
#                     if chunk:
#                         file.write(chunk)
#             else:
#                 file.write(response.content)
#         print(f"Tải xuống thành công tại: {file_path}")
#     else:
#         print(f"Không thể tải xuống file. Lỗi HTTP {response.status_code}")

# for url in download_links:
#     if "mp4" in url:
#         download_file(url, f"{LESSON}.mp4")
#     elif "vtt" in url:
#         download_file(url, "Subtitle.vtt")
#     elif "txt" in url:
#         download_file(url, "Transcript.txt")


In [21]:
# elems = driver.find_elements(By.CLASS_NAME, "cds-button-label")
# elem = next((e for e in elems if "Next" in e.text), elems[0])
# elem.click()


In [38]:
# download_html("https://www.coursera.org/learn/agile-atlassian-jira/supplement/Oy2DU/welcome", "i.html", "C:\\Users\\victo\\Downloads\\Coursera")

In [ ]:
# css_links = driver.find_elements(By.TAG_NAME, "link")
# for link in css_links:
#     href = link.get_attribute("href")
#     print(href)
#     if href and "css" in href:
#         css_content = driver.execute_script("return fetch(arguments[0]).then(response => response.text());", href)
#         css_file_name = href.split("/")[-1]
#         css_file_path = os.path.join("C:\\Users\\victo\\Downloads\\Coursera", css_file_name)
#         with open(css_file_path, "w", encoding="utf-8") as file:
#             file.write(css_content)


In [ ]:
# elem = driver.find_element(By.CLASS_NAME, "item-page-content")
# print(elem.text)
# link = elem.find_element(By.TAG_NAME, "a").get_attribute("href")
# link